In [1]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from joblib import dump, load

import sys


In [2]:

  # prediction path
INPUT_SPAN_FILE_PATH = "/data/edan/for_edan/cofie_spans.txt"

def embed_text(text, model):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
    input_ids = input_ids.to('cuda:1')
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    return last_hidden_states 

model_version = 'allenai/scibert_scivocab_uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version)
model.to('cuda:1')
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
input_file = open(INPUT_SPAN_FILE_PATH)

def get_words(input_file, model):
  word_list = []
  word_em_list = []
  count = 0
  for line in input_file:
    if count% 10000 == 0:
      print(count)
    # if count == 100000:
    #   break
    count += 1
    if line[:-1] not in word_list:
      word_list.append(line[:-1])
      word_em_list.append(embed_text(word_list[-1], model).mean(1).detach().cpu().numpy()[0])
  return word_list, word_em_list

word_list, word_em_list = get_words(input_file, model)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
dump(word_list, "wordlist.joblib")
dump(word_em_list, "wordemlist.joblib")

['wordemlist.joblib']

In [12]:
word_list = load("scripts/revised_wordlist.joblib")
word_em_list = load("scripts/revised_wordemlist.joblib")

In [14]:
word_list[1]

'long term imaging follow'

In [7]:
kmeans = load("training_kmeans.joblib")
predictions = kmeans.predict(word_em_list)

3


In [16]:
output_file = open("k_means_clusters_predict.tsv", "w")
for word, prediction in zip(word_list, predictions):
    output_file.write(word + "\t" + str(prediction) + "\n")

1248011
1248011
1248011
